## Import Libraries

In [21]:
import os
import wave 
import langid
#import openai
#import librosa

import sklearn
import pyaudio #python bindings for PortAudio the cross_platform audio i/o  library

import fasttext
import playsound
from gtts import gTTS
import librosa.display
from langdetect import detect
from pydub import AudioSegment
import pyttsx3 #text to speech
import matplotlib.pyplot as plt
import speech_recognition as sr #Speech to text
from translate import Translator
#from googletrans import Translator
from tempfile import NamedTemporaryFile
from IPython.display import Audio, display
from transformers import MarianMTModel, MarianTokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
#from transliterate import translit

c=0

In [10]:
# Ensure pydub can find the ffmpeg/ffprobe binary
AudioSegment.converter = "ffmpeg"  # path to ffmpeg binary

#pip install translate
#wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
'''/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
brew install ffmpeg
ffmpeg -version
#pip install speechrecognition
#pip install pyttsx3
#pip install pyttsx4
#brew install portaudio
#pip uninstall pyaudio
#pip install pyaudio

'''

'/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"\nbrew install ffmpeg\nffmpeg -version\n#pip install speechrecognition\n#pip install pyttsx3\n#pip install pyttsx4\n#brew install portaudio\n#pip uninstall pyaudio\n#pip install pyaudio\n\n'

## Used Functions

In [26]:
# choose_microphone
def choose_microphone():
    print("Available Microphones:")
    microphones = sr.Microphone.list_microphone_names()
    for i, mic in enumerate(microphones):
        print(f"{i}: {mic}")
    while True:
        choice = input("Enter the index or name of the microphone you want to use to record audio: ")
        if choice.isdigit():
            choice = int(choice)
            if 0 <= choice < len(microphones):
                print(f"The microphone used is {choice} : {microphones[choice]}.")
                return choice
        elif choice in microphones:
            print(f"The microphone used is {choice} : {microphones[choice]}.")
            return choice
        print("Invalid choice. Please enter a valid index or microphone name.")


# Function to translate "I'm Listening to you..." automatically
def translit_message(text,language):
    translator = Translator(to_lang=language)

    try:
        # Translate using translate library
        translation = translator.translate(text)
        return translation
    except Exception as e:
        return f"Language not supported for translation: {str(e)}"


# save input audio wave
def save_input_audio(audio, c, save_dir="audio_in_out/inputs_audio"):
    # Ensure the directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Write wave file
    sample_rate = 48000.0  # Hertz
    frames = audio.get_wav_data()
    wav_filename = os.path.join(save_dir, f"input_audio_{c}.wav")
    with wave.open(wav_filename, 'w') as obj:
        obj.setnchannels(1)  # Mono or 2
        obj.setsampwidth(2)
        obj.setframerate(sample_rate)
        obj.writeframes(frames)
    
    print(f"Audio has been saved to: {wav_filename}")
    return wav_filename


# save output audio wave
def save_output_audio(audio, c, save_dir="audio_in_out/outputs_audio"):
    # Ensure the directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Write wave file
    sample_rate = 48000.0  # Hertz
    frames = audio.get_wav_data()
    wav_filename = os.path.join(save_dir, f"output_audio_{c}.wav")
    with wave.open(wav_filename, 'w') as obj:
        obj.setnchannels(1)  # Mono or 2
        obj.setsampwidth(2)
        obj.setframerate(sample_rate)
        obj.writeframes(frames)
    
    print(f"Audio has been saved to: {wav_filename}")
    return wav_filename


# save the INPUT text output in a file 
def input_text_to_file(text, c, save_dir="text_in_out/inputs_text"):
    file_path = os.path.join(save_dir, f"input_text_{c}.txt")
    with open(file_path, "a") as f:
        f.write(text + "\n")
        print(text," was written in ",file_path)


# save the OUTPUT text output in a file 
def output_text_to_file(text, c, save_dir="text_in_out/outputs_text"):
    file_path = os.path.join(save_dir, f"output_text_{c}.txt")
    with open(file_path, "a") as f:
        f.write(text + "\n")
        print(text," was written in ",file_path)


# record Speech and convert it to Text as input
def record_speech_to_text(chosen_microphone,c,language):
    recognizer = sr.Recognizer()
    #get audio and save it
    with sr.Microphone(device_index=chosen_microphone) as source:
        recognizer.adjust_for_ambient_noise(source)  # Adjust for noise
        translation=translit_message("Hello I'm listening, how can I help you?",language)
        text_to_speech(translation, language) 
        print(translation,"...")
        '''if language=='en' :
            print("Listening...")
        elif language=='ar':
            print("...في الاستماع")
        elif language=='fr':
            print("écoute...")'''
        audio = recognizer.listen(source)
        # Save the captured audio to a WAV file
        wav_filename = save_input_audio(audio, c)

        # Play back the captured audio
        #play_audio(audio)
        playsound.playsound(wav_filename)  # Play the input wav file in recorded voice instead of speak it 

    #convert audio to text
    try:
        # Use Google's speech recognition with language detection
        text = recognizer.recognize_google(audio, language=language) # Use Google's speech recognition and Specify language parameter for Arabic ('ar')
        print("audio has been converted to : ",text)
        #input_text_to_file(text, c)
        return text
    except sr.UnknownValueError:
        print("Could not understand audio")
        return ""
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))
        return ""


# read the INPUT text from a file
def read_text_from_inputs(c,save_dir="text_in_out/inputs_text"):
    file_path = os.path.join(save_dir, f"input_text_{c}.txt")
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


# read the OUTPUT text from a file
def read_text_from_outputs(c,save_dir="text_in_out/outputs_text"):
    file_path = os.path.join(save_dir, f"output_text_{c}.txt")
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


# speak the text_output and save it in a file 
def text_to_speech(text, lang):
    if not text:
        translation = translit_message("There is no text_to_speech conversion.", lang)
        print(translation)
        return  # Return if there is no text to convert

    # Convert text to speech
    with NamedTemporaryFile(delete=False) as tmp_mp3:
        tts = gTTS(text=text, lang=lang)
        tts.save(tmp_mp3.name)
        tmp_mp3.flush()  # Ensure all data is written to the file

        # Play the speech
        playsound.playsound(tmp_mp3.name, True)



# speak the text_output and save it in a file 
def text_to_speach_input(text, lang, c, audio_save_dir="audio_in_out/inputs_audio"):
    if not text:
        translation=translit_message("There is no text_to_speech conversion.",lang)
        print(translation)
        return
        '''if lang == 'en':
            print("There is no text_to_speech conversion.")
        elif lang == 'ar':
            print(".لا يوجد نص للتحويل إلى كلام")
        elif lang == 'fr':
            print("Il n’y a pas de conversion texte-parole.")
        '''

    # Ensure the directory exists
    if not os.path.exists(audio_save_dir):
        os.makedirs(audio_save_dir)
        
    # Generate the filename for the wav file
    wav_filename = os.path.join(audio_save_dir, f"input_audio_{c}.wav")

    # Convert text to speech and save directly as wav using a temporary mp3 file
    with NamedTemporaryFile(delete=True) as tmp_mp3:
        tts = gTTS(text=text, lang=lang)
        tts.save(tmp_mp3.name)

        # Convert the mp3 file to wav
        sound = AudioSegment.from_mp3(tmp_mp3.name)
        sound.export(wav_filename, format="wav")


    # Play the wav file
    playsound.playsound(wav_filename)

    print(f"The response audio has been saved to: {wav_filename}")


# speak the text_output and save it in a file 
def text_to_speach_output(text, lang, c, audio_save_dir="audio_in_out/outputs_audio"):
    if not text:
        translation=translit_message("There is no text_to_speech conversion.",lang)
        print(translation)
        return
        '''if lang == 'en':
            print("There is no text_to_speech conversion.")
        elif lang == 'ar':
            print(".لا يوجد نص للتحويل إلى كلام")
        elif lang == 'fr':
            print("Il n’y a pas de conversion texte-parole.")
        '''

    # Ensure the directory exists
    if not os.path.exists(audio_save_dir):
        os.makedirs(audio_save_dir)
        
    # Generate the filename for the wav file
    wav_filename = os.path.join(audio_save_dir, f"output_audio_{c}.wav")

    # Convert text to speech and save directly as wav using a temporary mp3 file
    with NamedTemporaryFile(delete=True) as tmp_mp3:
        tts = gTTS(text=text, lang=lang)
        tts.save(tmp_mp3.name)

        # Convert the mp3 file to wav
        sound = AudioSegment.from_mp3(tmp_mp3.name)
        sound.export(wav_filename, format="wav")


    # Play the wav file
    playsound.playsound(wav_filename)

    print(f"The response audio has been saved to: {wav_filename}")


# speak the text_output from a file and save it in an audio 
def text_to_speach_output_from_file(c, audio_save_dir="audio_in_out/outputs_audio"):
    # Generate the filename for the wav file
    wav_filename = os.path.join(audio_save_dir, f"output_audio_{c}.wav")

    # Ensure the directory exists
    if not os.path.exists(audio_save_dir):
        os.makedirs(audio_save_dir)

    # Read the text from the file
    text = read_text_from_outputs(c)
    if not text:
        #translation=translit_message("There is no text_to_speech conversion.",language_code)
        #print(translation)
        print("There is no text_to_speech conversion.")
        '''if lang == 'en':
            print("There is no text_to_speech conversion.")
        elif lang == 'ar':
            print(".لا يوجد نص للتحويل إلى كلام")
        elif lang == 'fr':
            print("Il n’y a pas de conversion texte-parole.")'''
        return
    language_code, language = detect_language(text)
    print("text: ", text, " in ",language)


    # Convert text to speech and save directly as wav using a temporary mp3 file
    with NamedTemporaryFile(delete=True) as tmp_mp3:
        tts = gTTS(text=text,lang=language_code)#, 
        tts.save(tmp_mp3.name)

        # Convert the mp3 file to wav
        sound = AudioSegment.from_mp3(tmp_mp3.name)
        sound.export(wav_filename, format="wav")

    # Play the wav file
    playsound.playsound(wav_filename)

    print(f"The response audio has been saved to: {wav_filename}")



# Function to detect the language of the input text
fasttext_model = fasttext.load_model('lid.176.bin')# Pre-Trained model for language identification
# Language code to language name mapping
language_mapping = {
    "af": "Afrikaans",
    "ar": "Arabic",
    "bg": "Bulgarian",
    "bn": "Bengali",
    "ca": "Catalan",
    "cs": "Czech",
    "da": "Danish",
    "de": "German",
    "el": "Greek",
    "en": "English",
    "es": "Spanish",
    "et": "Estonian",
    "fa": "Persian",
    "fi": "Finnish",
    "fr": "French",
    "he": "Hebrew",
    "hi": "Hindi",
    "hr": "Croatian",
    "hu": "Hungarian",
    "id": "Indonesian",
    "it": "Italian",
    "ja": "Japanese",
    "ka": "Georgian",
    "ko": "Korean",
    "lt": "Lithuanian",
    "lv": "Latvian",
    "mk": "Macedonian",
    "ml": "Malayalam",
    "mr": "Marathi",
    "ms": "Malay",
    "nl": "Dutch",
    "no": "Norwegian",
    "pl": "Polish",
    "pt": "Portuguese",
    "ro": "Romanian",
    "ru": "Russian",
    "sk": "Slovak",
    "sl": "Slovenian",
    "sq": "Albanian",
    "sr": "Serbian",
    "sv": "Swedish",
    "sw": "Swahili",
    "ta": "Tamil",
    "te": "Telugu",
    "th": "Thai",
    "tl": "Tagalog",
    "tr": "Turkish",
    "uk": "Ukrainian",
    "ur": "Urdu",
    "vi": "Vietnamese",
    "zh": "Chinese",
    # Add more languages as needed
}
# Function to detect the language of the input text using fastText
def detect_language(text):
    # Remove newline characters from the input text
    text = text.replace('\n', ' ')
    predictions = fasttext_model.predict(text, k=1)  # k=1 returns the top prediction
    language_code = predictions[0][0].replace('__label__', '')
    language_name = language_mapping.get(language_code, "Unknown")
    return language_code, language_name

#'''
#play audio function
def play_audio(audio):
    audio_data = audio.get_wav_data()
    with wave.open("temp_audio.wav", "wb") as f:
        f.setnchannels(1)
        f.setsampwidth(pyaudio.PyAudio().get_sample_size(pyaudio.paInt16))
        f.setframerate(16000)
        f.writeframes(audio_data)
        
    # Open the temporary WAV file and play it
    p = pyaudio.PyAudio()
    wf = wave.open("temp_audio.wav", "rb")# read byte mode
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)
    data = wf.readframes(1024)
    while data:
        stream.write(data)
        data = wf.readframes(1024)
    stream.stop_stream()
    stream.close()

    p.terminate()#'''

## Try Functions

In [12]:
# choose_microphone
chosen_microphone = choose_microphone()#3 #

Available Microphones:
0: Built-in Microphone
1: Built-in Output
2: USB Audio Device
3: USB Audio Device
4: Soundflower (2ch)
5: Soundflower (64ch)
6: Screenflick Loopback
7: iShowU Audio Capture
8: Périphérique agrégé
9: Périphérique à sortie multiple
The microphone used is 3 : USB Audio Device.


In [13]:

# test audio:
#text_to_speech("Hello, how are you?", "en")

In [15]:
'''
# Function to translate "I'm Listening to you..." automatically
text="how are you I'm Listening to you...?"
print(translit_message(text,'ar'))  # Output: أنا أستمع إليك... (or similar in Arabic)
print(translit_message(text,'en'))  # Output: I'm Listening to you...
print(translit_message(text,'fr'))  # Output: Je vous écoute... (or similar in French)
print(translit_message(text,'it'))  # Output: 我在听你... (or similar in Chinese)'''


كيف حالك أنا أستمع إليك... ؟
how are you I'm Listening to you...?
comment tu fais je t'écoute... ?
come ti sto ascoltando...?


In [9]:
'''# save the INPUT text output in a file 
input_text_to_file("كم الساعة الان؟", 1)


# save the OUTPUT text output in a file 
output_text_to_file("الساعة الان الواحدة و النصف ",1)#'''

'# save the INPUT text output in a file \ninput_text_to_file("كم الساعة الان؟", 1)\n\n\n# save the OUTPUT text output in a file \noutput_text_to_file("الساعة الان الواحدة و النصف ",1)#'

In [25]:
# record Speech and convert it to Text as input
text = record_speech_to_text(chosen_microphone,6,'ar')
print("text recorded: ",text)#'''

مرحبًا، أنا أستمع، كيف يمكنني مساعدتك ؟ ...
Audio has been saved to: audio_in_out/inputs_audio/input_audio_6.wav


KeyboardInterrupt: 

In [25]:
'''# read the INPUT text from a file
input=read_text_from_inputs(0)
print("input: ",input)

# read the OUTPUT text from a file
output=read_text_from_outputs(13)
print("output: ",output)

#language_code, language = detect_language(output) #get language of output
#text_to_speach_output(output, language_code, c) # speak the text_output and save it in a file '''


output:  كم عدد ايام الاسبوع


The response audio has been saved to: audio_in_out/outputs_audio/output_audio_13.wav


In [9]:
'''detected language
prompt1 = "Cómo estás hoy"
prompt2 = "Bonjour Speechy"
prompt3 = "Hello Speechy how are you"
prompt4 = "مرحبا سبيشي"

code1, name1 = detect_language(prompt1)
code2, name2 = detect_language(prompt2)
code3, name3 = detect_language(prompt3)
code4, name4 = detect_language(prompt4)

print(f"The detected language1 is: {code1} ({name1})")
print(f"The detected language2 is: {code2} ({name2})")
print(f"The detected language3 is: {code3} ({name3})")
print(f"The detected language4 is: {code4} ({name4})")'''

'# Example usage\nprompt1 = "Cómo estás hoy"\nprompt2 = "Bonjour Speechy"\nprompt3 = "Hello Speechy how are you"\nprompt4 = "مرحبا سبيشي"\n\ncode1, name1 = detect_language(prompt1)\ncode2, name2 = detect_language(prompt2)\ncode3, name3 = detect_language(prompt3)\ncode4, name4 = detect_language(prompt4)\n\nprint(f"The detected language1 is: {code1} ({name1})")\nprint(f"The detected language2 is: {code2} ({name2})")\nprint(f"The detected language3 is: {code3} ({name3})")\nprint(f"The detected language4 is: {code4} ({name4})")'

In [8]:
# General Test
c+=1

#GET INPUT
input_text = record_speech_to_text(chosen_microphone,c,'ar')# record Speech and convert it to Text as input
print("text recorded: ",input_text)
input_text_to_file(input_text, c) #save input to file in inputs_folder 
language_code, language = detect_language(input_text)
print("input: ", input_text, " in ",language)
playsound.playsound(f"audio_in_out/inputs_audio/input_audio_{c}.wav")  # Play the input wav file in recorded voice instead of speak it 
#text_to_speach_input(input_text, language_code, c)




#GENERATE OUTPUT
gen_output=read_text_from_inputs(c) #geting text from inputs insted of generating a new output based on the input    #(in futer) generating a new output based on the input   
output_text_to_file(gen_output, c)  #save gen_output to a file in outputs_folder 



#GIVE OUTPUT
#output=read_text_from_outputs(c)    #get output from outputs_folder
#language_code, language = detect_language(output) #get language of output
#text_to_speach_output(output, language_code, c) # speak the text_output and save it in a file 
text_to_speach_output_from_file(c)# speak the text_output from a file and save it in an audio '''

مرحبًا، أنا أستمع، كيف يمكنني مساعدتك ؟ ...


KeyboardInterrupt: 

In [ ]:
'''plt.figure(figsize=(14, 5))
librosa.display.waveshow(data, sr=sr)'''

In [ ]:
# spectrogram
'''X = librosa.stft(data) #short-time Fourier transform (STFT)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar()'''

In [ ]:
#Features Analysis, Spectral Centroid
'''import sklearn.preprocessing


spectral_centroids = librosa.feature.spectral_centroid(y=data, sr=sr)[0]
print(spectral_centroids.shape)

#computing the time variable for visualization
plt.figure(figsize=(14,5))
frames = range(len(spectral_centroids))
t= librosa.frames_to_time(frames)

#normalizing the spectral centroids for visualisation
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=0)
#plotting the spectral centroids along the waveform
librosa.display.waveshow(data, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='b')'''

In [ ]:
'''import speech_recognition as sr

def list_microphones():
    for i, mic in enumerate(sr.Microphone.list_microphone_names()):
        print(f"{i}: {mic}")

def choose_microphone():
    print("Please choose a microphone from the list:")
    list_microphones()
    mic_index = int(input("Enter the microphone index: "))
    return mic_index

def record_text(mic_index, recognizer):
    with sr.Microphone(device_index=mic_index) as source:
        print("Say something!")
        audio = recognizer.listen(source)
    return audio

def main():
    recognizer = sr.Recognizer()
    chosen_microphone = choose_microphone()

    try:
        audio = record_text(chosen_microphone, recognizer)
        text = recognizer.recognize_google(audio, language='ar')
        print("You said: " + text)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")

if __name__ == "__main__":
    main()'''


## 1. Detect Language:

In [ ]:
# Function to detect the language of the input text
# Load the pre-trained fastText model for language identification
fasttext_model = fasttext.load_model('lid.176.bin')# Pre-Trained model for language identification

# Function to detect the language of the input text using fastText
def detect_language(text):
    predictions = fasttext_model.predict(text, k=1)  # k=1 returns the top prediction
    language_code = predictions[0][0].replace('__label__', '')
    return language_code


In [ ]:
'''prompt1 = "Cómo estás hoy"
prompt2 = "Bonjour Speechy"
prompt3 = "Hello Speechy how are you"
prompt4 = "مرحبا سبيشي"

print(f"The detected language1 is: {detect_language(prompt1)}")
print(f"The detected language2 is: {detect_language(prompt2)}")
print(f"The detected language3 is: {detect_language(prompt3)}")
print(f"The detected language4 is: {detect_language(prompt4)}")'''

In [ ]:

# Function to convert speech to text
def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return ""
    except sr.RequestError as e:
        return ""

# Function to record audio and save it to a file
def record_audio(output_file, duration=5):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Recording audio...")
        audio = recognizer.listen(source, phrase_time_limit=duration)
        with open(output_file, "wb") as f:
            f.write(audio.get_wav_data())
    print(f"Audio recorded and saved as {output_file}")

# Record audio
audio_file = "recorded_audio.wav"
record_audio(audio_file)

# Convert the recorded audio to text
text = speech_to_text(audio_file)
if text:
    print(f"Transcribed Text: {text}")
    # Detect the language of the transcribed text
    language = detect_language(text)
    print(f"Detected Language: {language}")
else:
    print("Could not transcribe the audio.")

## 2. Generate Response:


In [ ]:
# Assuming we have a function `generate_response` that uses GPT-4
'''# Function to generate a response using a multilingual model
'' 'def generate_response(prompt, language):
    # For simplicity, assume we're using a multilingual variant of GPT-4 or similar
    # Load a pre-trained model and tokenizer
    model_name = 'gpt-4-multilingual'
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    print(tokenizer)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    # Encode the input prompt
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # Generate a response
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response'' '



# Function to generate a response using a multilingual translation model
def generate_response(prompt, language):
    # Load the pre-trained model and tokenizer for translation
    model_name = 'Helsinki-NLP/opus-mt-mul-en'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    print(tokenizer)

    ' ''model = MarianMTModel.from_pretrained(model_name)

    # Translate prompt to English for processing
    translated = tokenizer.encode(prompt, return_tensors='pt')
    translated_text = model.generate(translated)
    translated_prompt = tokenizer.decode(translated_text[0], skip_special_tokens=True)

    # Here you would use a GPT model to generate a response in English
    # For simplicity, let's mock this step with a simple response
    response_in_english = f"This is a generated response for: {translated_prompt}"

    # Translate the response back to the original language
    back_translated = tokenizer.encode(response_in_english, return_tensors='pt')
    back_translated_text = model.generate(back_translated)
    final_response = tokenizer.decode(back_translated_text[0], skip_special_tokens=True)
    
    return final_response'' '




# Example usage
prompt = "hello Speechy"
language = detect_language(prompt)
print(f"hello Speechy is in {language}")

generate_response(prompt, language)
#response = generate_response(prompt, language)
#print(response)'''

## 3. googletrans  Translator

In [ ]:
'''translator = Translator()

# Translate prompt to English
translated_prompt = translator.translate(prompt, src=language, dest='en').text

# Generate response in English
response_in_english = generate_response(translated_prompt)

# Translate response back to the detected language
final_response = translator.translate(response_in_english, src='en', dest=language).text
print(final_response)
'''

In [ ]:
'''
import os
import openai
import langid
from translate import Translator

# Set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Function to detect the language of the input text using langid
def detect_language(text):
    lang, _ = langid.classify(text)
    return lang

# Function to translate text using the translate library
def translate_text(text, from_lang, to_lang):
    translator = Translator(from_lang=from_lang, to_lang=to_lang)
    return translator.translate(text)

# Function to generate a response using OpenAI's GPT-3/4
def generate_response(prompt, language):
    # Translate the prompt to English if it's not in English
    if language != 'en':
        prompt_in_english = translate_text(prompt, language, 'en')
    else:
        prompt_in_english = prompt
    
    # Generate a response using OpenAI's GPT-3/4
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt_in_english,
        max_tokens=150
    )
    
    response_in_english = response.choices[0].text.strip()
    
    # Translate the response back to the original language if it's not in English
    if language != 'en':
        final_response = translate_text(response_in_english, 'en', language)
    else:
        final_response = response_in_english
    
    return final_response

# Example usage
prompts = ["Cómo estás hoy", "bonjour Speechy", "Hello Speechy how are you", "مرحبا سبيشي"]
for i, prompt in enumerate(prompts, 1):
    language = detect_language(prompt)
    print(f"Detected language for prompt {i}: {language}")
    response = generate_response(prompt, language)
    print(f"Response {i}: {response}")
'''

In [ ]:
#### INPUT
#select a microphone #chosen_microphone = choose_microphone()
#get_input_audio 1
#audio_to_text_input 1
#save_text_to_file 1

#### OUTPUT
#generate_output_text 0
#save_text_to_file 1
#text_to_speech_autput 1
#play_output 1

In [ ]:
#play audio function
def play_audio(audio):
    audio_data = audio.get_wav_data()
    with wave.open("temp_audio.wav", "wb") as f:
        f.setnchannels(1)
        f.setsampwidth(pyaudio.PyAudio().get_sample_size(pyaudio.paInt16))
        f.setframerate(16000)
        f.writeframes(audio_data)
        
    # Open the temporary WAV file and play it
    p = pyaudio.PyAudio()
    wf = wave.open("temp_audio.wav", "rb")# read byte mode
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)
    data = wf.readframes(1024)
    while data:
        stream.write(data)
        data = wf.readframes(1024)
    stream.stop_stream()
    stream.close()

    p.terminate()

In [ ]:
# Load the pre-trained fastText model for language identification
fasttext_model = fasttext.load_model('lid.176.bin')

# Function to detect the language of the input text using fastText
def detect_language(text):
    predictions = fasttext_model.predict(text, k=1)  # k=1 returns the top prediction
    language_code = predictions[0][0].replace('__label__', '')
    return language_code

# Function to convert speech to text
def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return ""
    except sr.RequestError as e:
        return ""

# Function to record audio and save it to a file
def record_audio(output_file, duration=5):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Recording audio...")
        audio = recognizer.listen(source, phrase_time_limit=duration)
        with open(output_file, "wb") as f:
            f.write(audio.get_wav_data())
    print(f"Audio recorded and saved as {output_file}")

# Record audio
audio_file = "/recorded_audio.wav"
record_audio(audio_file)

'''# Convert the recorded audio to text
text = speech_to_text(audio_file)
if text:
    print(f"Transcribed Text: {text}")
    # Detect the language of the transcribed text
    language = detect_language(text)
    print(f"Detected Language: {language}")
else:
    print("Could not transcribe the audio.")'''


In [ ]:
import os
import fasttext
import speech_recognition as sr
from gtts import gTTS
from playsound import playsound
from pydub import AudioSegment
from tempfile import NamedTemporaryFile

# Load the pre-trained fastText model for language identification
fasttext_model = fasttext.load_model('lid.176.bin')

# Function to detect the language of the input text using fastText
def detect_language(text):
    predictions = fasttext_model.predict(text, k=1)  # k=1 returns the top prediction
    language_code = predictions[0][0].replace('__label__', '')
    return language_code

# Function to convert speech to text
def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return ""
    except sr.RequestError as e:
        return ""

# Function to record audio and save it to a file
def record_audio(output_file, duration=5):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Recording audio...")
        audio = recognizer.listen(source, phrase_time_limit=duration)
        with open(output_file, "wb") as f:
            f.write(audio.get_wav_data())
    print(f"Audio recorded and saved as {output_file}")

# Function to save text to a file
def save_text_to_file(text, filename):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)

# Function to read text from a specific paragraph in a file
def read_paragraph_from_file(filename, paragraph_num):
    with open(filename, "r", encoding="utf-8") as f:
        paragraphs = f.read().split("\n\n")
        if paragraph_num < len(paragraphs):
            return paragraphs[paragraph_num]
        else:
            return ""

# Function to convert text to speech and save it as a wav file
def text_to_speech(text, lang, output_file):
    with NamedTemporaryFile(delete=True) as tmp_mp3:
        tts = gTTS(text=text, lang=lang)
        tts.save(tmp_mp3.name)
        sound = AudioSegment.from_mp3(tmp_mp3.name)
        sound.export(output_file, format="wav")

# Function to play the audio file
def play_audio(file):
    playsound(file)

# Main process
def main():
    input_audio_file = "input_audio.wav"
    transcribed_text_file = "transcribed_text.txt"
    output_text_file = "output_text.txt"
    output_audio_file = "output_audio.wav"

    # INPUT
    # Record audio
    record_audio(input_audio_file)

    # Convert audio to text
    transcribed_text = speech_to_text(input_audio_file)
    print(f"Transcribed Text: {transcribed_text}")

    # Save transcribed text to a file
    save_text_to_file(transcribed_text, transcribed_text_file)

    # OUTPUT
    # Generate output text (for simplicity, we will use the same transcribed text)
    output_text = transcribed_text

    # Save output text to a file
    save_text_to_file(output_text, output_text_file)

    # Detect language of the output text
    language = detect_language(output_text)
    print(f"Detected Language: {language}")

    # Convert text to speech and save it as a wav file
    text_to_speech(output_text, language, output_audio_file)

    # Play the output audio
    play_audio(output_audio_file)
    print(f"The response audio has been saved to: {output_audio_file}")

if __name__ == "__main__":
    main()
